In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter,
import matplotlib.ticker as ticker


from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

import seaborn as sns
import autograd.numpy as np
from autograd import grad, elementwise_grad,jacobian, hessian
import pandas as pd
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.utils import resample
from sklearn.model_selection import KFold, cross_val_score
from sklearn.datasets import load_breast_cancer
plt.rcParams['font.size'] = 14


In [2]:
def Sigmoid(y):
    return np.exp(y)/(1+np.exp(y))
def Identity(y):
    return y

def RELU(y):
    return np.maximum(0,y)

def lexyRelu(y):
    return np.maximum(0.01*y,y)

def Set_weights_and_bias(n_in,n_hidden,n_out):
#### Setting hiden weights
    W_hidden = np.random.randn(n_in, n_hidden)
    b_hidden = np.zeros(n_hidden) +0.01
#### setting output weights
    W_out = np.random.randn(n_hidden, n_out)
    b_out = np.zeros(n_out) +0.01
    return W_hidden, W_out, b_hidden, b_out

#### From lecture notes
def feed_forward_train(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function):
#### Hidden attac
    z_h = X@W_hidden + b_hidden
    a_h = activation_function(z_h)
#### output attac
    z_o = a_h@W_out + b_out
    a_o = output_function(z_o)
    return a_h, a_o, z_h,z_o


def feed_forward(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function):
#### Hidden attac
    z_h = X@W_hidden + b_hidden
    a_h = activation_function(z_h)
#### output attac
    z_o = a_h@W_out + b_out
    a_o = output_function(z_o)
    return a_o[0]

def u(x):
    return np.sin(np.pi*x)

def g_trial(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function):
    x,t = X[0],X[1]
    return (1-t)*u(x) + x*(1-x)*t*feed_forward(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)

#### From notes
def cost_function(x,t,W_hidden, W_out, b_hidden, b_out,activation_function, output_function):
    cost_sum = 0
    
    g_t_jac_fun = jacobian(g_trial)
    g_t_hessian_fun = hessian(g_trial)
    
    for x_ in x:
        for t_ in t:
            X = np.array([x_,t_])
            g_t = g_trial(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            g_t_jac = g_t_jac_fun(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            g_t_hessian =g_t_hessian_fun(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
           # print(g_t)
            g_t_dt = g_t_jac[1]
            g_t_d2x = g_t_hessian[0][0]
            
            error = g_t_dt -g_t_d2x
            cost_sum += error**2
    return cost_sum/(np.size(x)*np.size(t))

### from notes, altered to fitt our setup
def solution(x,t,N_hidden,eta, activation_function ,output_function, epoch,Minibach):
    W_hidden, W_out, b_hidden, b_out = Set_weights_and_bias(2,N_hidden,1)
 #   print('Initial cost: ', cost_function(x,t,W_hidden, W_out, b_hidden, b_out,activation_function, output_function))
    
    cost_func_wh_grad = elementwise_grad(cost_function,2)
    cost_func_bh_grad = elementwise_grad(cost_function,4)
    cost_func_wo_grad = elementwise_grad(cost_function,3)
    cost_func_bo_grad = elementwise_grad(cost_function,5)
    MiniBachSize =int(np.size(x)/Minibach)
    for e in range(epoch):
        for j in range(Minibach):
            miniBach = np.random.randint(Minibach)
            miniBachMin, miniBachMax = MiniBachSize * miniBach,(MiniBachSize) * (miniBach+1)
            x_bach,t_bach = x[miniBachMin:miniBachMax],t[miniBachMin:miniBachMax]
            W_hidden -= eta*cost_func_wh_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            W_out -= eta*cost_func_wo_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            b_hidden -= eta*cost_func_bh_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            b_out -= eta*cost_func_bo_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
      #  print("finished iteration number: ", e)
   
   # print('Final cost: ', cost_function(x,t,W_hidden, W_out, b_hidden, b_out,activation_function, output_function))
    return W_hidden, W_out, b_hidden, b_out

#### This one is nice when we want to study how the MSE converges
def Solution2(x,t,N_hidden,eta, epoch,Minibach, W_hidden, W_out, b_hidden, b_out):
    cost_func_wh_grad = elementwise_grad(cost_function,2)
    cost_func_bh_grad = elementwise_grad(cost_function,4)
    cost_func_wo_grad = elementwise_grad(cost_function,3)
    cost_func_bo_grad = elementwise_grad(cost_function,5)
    activation_function = Sigmoid
    output_function = Identity
    MiniBachSize =int(np.size(x)/Minibach)
    for e in range(epoch):
        for j in range(Minibach):
            miniBach = np.random.randint(Minibach)
            miniBachMin, miniBachMax = MiniBachSize * miniBach,(MiniBachSize) * (miniBach+1)
            x_bach,t_bach = x[miniBachMin:miniBachMax],t[miniBachMin:miniBachMax]
            W_hidden -= eta*cost_func_wh_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            W_out -= eta*cost_func_wo_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            b_hidden -= eta*cost_func_bh_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
            b_out -= eta*cost_func_bo_grad(x_bach,t_bach,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
      #  print("finished iteration number: ", e)
   
   # print('Final cost: ', cost_function(x,t,W_hidden, W_out, b_hidden, b_out,activation_function, output_function))
    return W_hidden, W_out, b_hidden, b_out


In [ ]:
##### We have changed how we test and train. The train is done on a random unsorted grid, while the test is done on a fixed grid
##### This square finds the 
x = np.random.uniform(0,1,6)
t = np.random.uniform(0,1,6)

t_test = np.linspace(0,1,50)
x_test = np.linspace(0,1,50)

etas = np.logspace(-4,1,4)


activation_functions = [Sigmoid,RELU,lexyRelu,Identity]
output_function = Identity
epoch,Minibach = 100, 3
N_hidden = 32
N_straps = 10

error_list = np.zeros((np.size(etas),np.size(activation_functions)))
##### Training our network :)
k,l= 0,0
for eta in etas:
    for activation_function in activation_functions:
        g_error = 0
        print("Doing the strap:)")
        for counter in range(N_straps):
            W_hidden, W_out, b_hidden, b_out = solution(x,t,N_hidden,eta, activation_function ,output_function, epoch,Minibach)
            error = 0
            u_t = np.zeros((50,50))
            i,j= 0,0
            for x_0 in x_test:
                for t_0 in t_test:
                    X_0 = np.array([x_0,t_0])
                    u_t[j,i] = g_trial(X_0,W_hidden, W_out, b_hidden, b_out,Sigmoid, Identity)
                    error += (u_t[j,i]-u(x_0)*np.exp(-np.pi**2*t_0))**2
                    j+=1
                j=0
                i+=1
   #         plt.imshow(u_t,origin='lower')
   #         plt.show()
            error/= (np.size(t_test)*np.size(x_test))
            g_error += error
           # print(error)
        g_error /= N_straps
        error_list[k,l] += g_error
        l+=1
    k+=1
    l= 0 

tick = ticker.ScalarFormatter(useOffset=False, useMathText=True)
tick.set_powerlimits((0,0))

tx = [u"${}$".format(tick.format_data(x)) for x in [1,2,3,4]]
ty = [u"${}$".format(tick.format_data(x)) for x in etas]

#lambdas_sea = [lambdas[i] for i in range(len(lambdas))]
#etas_sea =  [etas[i] for i in range(len(etas))]

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(data=error_list,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
ax.set_xlabel(r'Activation Function')
ax.set_ylabel(r'$\eta$')
plt.tight_layout()
plt.savefig(f"Test_Accuracy_Single_hidden_layer.pdf")
plt.show()

Doing the strap:)


In [ ]:
### Solution as function of epochs
x = np.random.uniform(0,1,10)
t = np.random.uniform(0,1,10)

t_test = np.linspace(0,1,50)
x_test = np.linspace(0,1,50)

N_hidden = 32
eta = 0.01
epoch = 10
Minibach = 5

epoch_List = np.linspace(10,1000,100)
error_list = np.zeros_like(epoch_List)

W_hidden, W_out, b_hidden, b_out = Set_weights_and_bias(2,N_hidden,1)
for i in range(100):
    W_hidden, W_out, b_hidden, b_out = Solution2(x,t,N_hidden,eta, epoch,Minibach, W_hidden, W_out, b_hidden, b_out)
    error = 0
    u_t = np.zeros((50,50))
    l,j= 0,0
    for x_0 in x_test:
        for t_0 in t_test:
            X_0 = np.array([x_0,t_0])
            u_t[j,l] = g_trial(X_0,W_hidden, W_out, b_hidden, b_out,Sigmoid, Identity)
            error += (u_t[j,l]-u(x_0)*np.exp(-np.pi**2*t_0))**2
            j+=1
        j=0
        l+=1
#         plt.imshow(u_t,origin='lower')
#         plt.show()
    error/= (np.size(t_test)*np.size(x_test))
    error_list[i] = error
    print(i)
    
plt.plot(epoch_List,error_list)
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.tight_layout()
plt.savefig('MSE as a fucntin of epochs.pdf')
plt.show()

plt.plot(x_test, u_t[0,:],label=r'$t=0$')
plt.plot(x_test, u_t[9,:],label= r'$t=$' + str(t_test[9]))
plt.plot(x_test, u_t[19,:],label= r'$t=$' + str(t_test[19]))
plt.plot(x_test, u_t[29,:],label= r'$t=$' + str(t_test[29]))
plt.plot(x_test, u_t[39,:],label= r'$t=$' + str(t_test[39]))
plt.plot(x_test, u_t[49,:],label= r'$t=$' + str(t_test[49]))
plt.legend()
plt.xlabel(r'$x$')
plt.ylabel('MSE')
plt.show()




